# README
## Options:
### Pretrained model:
- word2vec-ruscorpora-300	
- glove-twitter-100	
### Tokenization
- 

In [ ]:
pd.set_option('display.max_rows', 400)
pd.set_option('display.max_columns', None)


# Data Import

In [5]:
import pandas as pd
import numpy as np

In [11]:
import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
import re
from nltk.corpus import stopwords as stpdfa
from nltk.stem.porter import PorterStemmer

import gensim
import gensim.models.word2vec as w2v
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [6]:
raw_data = pd.read_csv("./data/_PHEME_text.csv")

In [9]:
print(raw_data.shape)
raw_data.sample(5)

(5802, 2)


,text,Event
1080,"#ICYMI, Christopher Hitchens on the case for m...",charliehebdo
4120,DEVELOPING NEWS: Soldier shot at War Memorial....,ottawashooting
5004,RT @tomsteinfort: Terrifying photo of hostages...,sydneysiege
2305,#Ferguson chief said the officer was unaware o...,ferguson
4898,BREAKING: 2 people have run out of Sydney buil...,sydneysiege


In [76]:
raw_data.text.head(5)

0                                                       BREAKING: Armed man takes hostage in kosher grocery east of Paris http://t.co/PBs3sMwhLt
1                                                                                          #CharlieHebdo killers dead, confirmed by gendarmerie.
2       Top French cartoonists Charb, Cabu, Wolinski, Tignous confirmed among dead in #Paris #CharlieHebdo attack. Editor is critically wounded.
3    Police have surrounded the area where the #CharlieHebdo attack suspects are believed to be: http://t.co/3tGXEIX4F2\nhttps://t.co/aBSezf2QWS
4                          PHOTO: Armed gunmen face police officers near #CharlieHebdo HQ in Paris http://t.co/3Jsosc7yl3 http://t.co/iOpVNO6Iq0
Name: text, dtype: object

# Preprocessing

# Tokenization

In [65]:
from nltk.tokenize import TweetTokenizer
tweet_tokenizer = TweetTokenizer()
tweet_tokens = []
for sent in raw_data.text:
    # print(tweet_tokenizer.tokenize(sent))
    tweet_tokens.append([tweet_tokenizer.tokenize(sent)])
    # tweet_tokens.append(tweet_tokenizer.tokenize(sent))
df_tokens = pd.DataFrame(tweet_tokens, columns=['token'])

In [75]:
df_tokens.head(5)

,token
0,"[BREAKING, :, Armed, man, takes, hostage, in, kosher, grocery, east, of, Paris, http://t.co/PBs3sMwhLt]"
1,"[#CharlieHebdo, killers, dead, ,, confirmed, by, gendarmerie, .]"
2,"[Top, French, cartoonists, Charb, ,, Cabu, ,, Wolinski, ,, Tignous, confirmed, among, dead, in, #Paris, #CharlieHebdo, attack, ., Editor, is, critically, wounded, .]"
3,"[Police, have, surrounded, the, area, where, the, #CharlieHebdo, attack, suspects, are, believed, to, be, :, http://t.co/3tGXEIX4F2, https://t.co/aBSezf2QWS]"
4,"[PHOTO, :, Armed, gunmen, face, police, officers, near, #CharlieHebdo, HQ, in, Paris, http://t.co/3Jsosc7yl3, http://t.co/iOpVNO6Iq0]"


In [73]:
pd.set_option('display.max_rows', 400)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 400)

# Word2Vec

## Downloading Pretrained model for Gensim

## Fetching pretrained Model and Convert the Raw Text

In [27]:
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
# corpus = api.load('text8')
# wv = api.load('word2vec-google-news-300')
# fasttext-wiki-news-subwords-300'
#  'glove-twitter-200',
model = api.load('glove-twitter-100')

In [20]:
# import inspect
# print(inspect.getsource(wv.__class__))

In [ ]:
# model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True) #without *norm_only* param

In [28]:
dog = model['dog']
print(dog.shape)
print(dog[:10])

(100,)
[ 0.50779  -1.0274    0.48136  -0.09417   0.44837  -0.52291   0.51498
 -0.038927  0.35867  -0.065994]


In [42]:
w2v_object = model.wv
w2v_vectors = w2v_object.vectors # here you load vectors for each word in your model
w2v_indices = {word: w2v_object.vocab[word].index for word in w2v_object.vocab} # here you load indices - with whom you can find an index of the particular word in your model 

In [ ]:
def vectorize(line): 
    words = []
    for word in line: # line - iterable, for example list of tokens 
        try:
            w2v_idx = w2v_indices[word]
        except KeyError: # if you does not have a vector for this word in your w2v model, continue 
            continue
        words.append(list(w2v_vectors[w2v_idx]))
        if not word:
            words.append(None)

        if len(line) > len(words):
            continue
    return np.asarray(words)

In [67]:
print("Tweet 1: ", raw_data['text'][1])
print("Indice of '{}': {}".format(df_tokens['token'][1][0], w2v_indices[df_tokens['token'][1][0]]))
# print("Indice of '{}': {}".format(raw_data['text_token'][1][0], w2v_vectors[w2v_indices[raw_data['text_token'][1][0]]]))
# print("Indice of '{}': {}".format(raw_data['text_token'][1][1], w2v_indices[raw_data['text_token'][1][1]]))
# print("Indice of '{}': {}".format(raw_data['text_token'][1][1], w2v_vectors[w2v_indices[raw_data['text_token'][1][1]]]))
# print("\nVector of the first headline:\n", vectorize(raw_data['text_token'][1]))

Tweet 1:  #CharlieHebdo killers dead, confirmed by gendarmerie.


KeyError: '#CharlieHebdo'